In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup ## use in webscraping 

In [2]:
## headers={'User-Agent':'Mozilla/5.0 (Windows NT 6.3; Win 64 ; x64) Apple WeKit /537.36(KHTML , like Gecko) Chrome/80.0.3987.162 Safari/537.36'} -requests.get('url',headers=headers).text
## webpage = requests.get('https://internshala.com/internships/page-1/',headers = headers).text


webpage = requests.get('https://internshala.com/internships/page-1/').text

In [3]:
soup = BeautifulSoup(webpage,'lxml')

In [4]:
internshala = soup.find_all('div',class_='internship_meta')
len(internshala)

50

In [5]:
internshala = soup.find_all('div', class_='internship_meta')
internshala

[<div class="internship_meta">
 <div class="internship-heading-container">
 <div class="company">
 <h3 class="job-internship-name">
                     Charity Drive                </h3>
 <div class="heading_6 company_name">
 <div class="company_and_premium">
 <p class="company-name">
                             Hamari Pahchan NGO                        </p>
 <div class="actively-hiring-badge">Actively hiring</div>
 </div>
 </div>
 </div>
 <div class="internship_logo">
 <img alt="Hamari Pahchan NGO" src="https://internshala-uploads.internshala.com/logo%2F62eb9d2f310b41659608367.png.webp"/>
 </div>
 </div>
 <div class="individual_internship_details individual_internship_internship">
 <div class="detail-row-1">
 <div class="row-1-item locations">
 <i class="ic-16-home"></i>
 <span>
 <a>Work from home</a> </span>
 </div>
 <div class="row-1-item">
 <i class="ic-16-calendar"></i>
 <span>1 Week</span>
 </div>
 <div class="row-1-item">
 <i class="ic-16-money"></i>
 <span class="stipend">20%

In [6]:
len(internshala)

50

In [7]:
final = pd.DataFrame()

for j in range(1, 11):  
    url = 'https://internshala.com/internships/page-{}/'.format(j)
    webpage = requests.get(url).text
    soup = BeautifulSoup(webpage, 'lxml')
    internshala = soup.find_all('div', class_='internship_meta')

role = []
company_name = []
hiring_badge = []
location = []
duration = []
stipend = []
for i in internshala:
    role.append(i.find('h3').text.strip())
    company_name.append(i.find('p', class_='company-name').text.strip())
    hiring_badge.append(i.find('div',class_='actively-hiring-badge').text.strip())
    location.append(i.find_all('div',class_='row-1-item')[0].text.strip())
    duration.append(i.find_all('div',class_='row-1-item')[1].text.strip())
    stipend.append(i.find_all('div',class_='row-1-item')[2].text.strip())
d={'role':role, 'company_name':company_name, 'hiring_badge':hiring_badge, 'location':location, 'duration':duration, 'stipend':stipend}

df = pd.DataFrame(d)

In [8]:
internshala[0].find_all('div',class_='row-1-item')[1].text.strip()

'6 Months'

In [9]:
def scrape_internshala(page_number):
    """
    Scrapes internship data from a specific page of Internshala.

    Args:
        page_number (int): The page number to scrape.

    Returns:
        pandas.DataFrame: A DataFrame containing the scraped internship data.
    """

    url = 'https://internshala.com/internships/page-{page_number}/'
    response = requests.get(url)
    response.raise_for_status()  # Raise an exception if the request failed

    soup = BeautifulSoup(response.content, 'html.parser')
    internships = soup.find_all('div', class_='internship_meta')

    data = []
    for internship in internships:
        role = internship.find('h3').text.strip()
        company_name = internship.find('p', class_='company-name').text.strip()
        hiring_badge = internship.find('div', class_='actively-hiring-badge')
        hiring_badge = hiring_badge.text.strip() if hiring_badge else None
        location = internship.find_all('div', class_='row-1-item')[0].text.strip()
        duration = internship.find_all('div', class_='row-1-item')[1].text.strip()
        stipend = internship.find_all('div', class_='row-1-item')[2].text.strip()
        offering = internship.find_all('div',class_='status-li')
        offering = offering[0].text.strip() if offering else None
        
        data.append({
            'role': role,
            'company_name': company_name,
            'hiring_badge': hiring_badge,
            'location': location,
            'duration': duration,
            'stipend': stipend,  # Added a comma here
            'offering_job':offering
        })

    return pd.DataFrame(data)

# Scrape data from pages 1 to 50
df = pd.concat([scrape_internshala(page) for page in range(1, 51)], ignore_index=True)

print(df)

                                    role  \
0                          Charity Drive   
1                   Video Editing/Making   
2           Business Development (Sales)   
3                       Brand Management   
4                        Market Research   
...                                  ...   
2495  International Business Development   
2496             Film Assistant Director   
2497                         Telecalling   
2498        Business Development (Sales)   
2499        Business Development (Sales)   

                              company_name     hiring_badge  \
0                       Hamari Pahchan NGO  Actively hiring   
1                              Bright Rise  Actively hiring   
2                                   FoodPe  Actively hiring   
3                       The SkinTech India  Actively hiring   
4                CavinKare Private Limited  Actively hiring   
...                                    ...              ...   
2495                     Image

In [10]:
df.shape

(2500, 7)

In [11]:
df = pd.DataFrame(df)


KeyError: 'stipend(per month)'

In [ ]:
df

In [ ]:
df.to_csv('internshala_data.csv', index=False)

In [ ]:
df = df.rename(columns={'stipend(per month)': 'stipend_perMonth'}) 

In [ ]:
df

In [ ]:
print(df.duplicated().sum())  # Count of duplicate rows

In [ ]:
df = df.drop_duplicates()

In [ ]:
df
df['stipend(per month)'] = df['stipend(per month)'].str.replace('/month','')

In [ ]:
df

In [ ]:
df.to_csv('internshala_data.csv', index=False)

In [ ]:
null_counts = df.isnull().sum()

In [ ]:
null_counts

In [ ]:
df = df.fillna('Not hiring')

In [ ]:
df

In [ ]:
df['location'] = df['location'].str.strip()

In [ ]:
df= df['location']

In [ ]:
df